In [3]:
import numpy as np
import pandas as pd
import math
from math import sin, cos, pi, factorial, exp
import random
from scipy import stats
%matplotlib inline
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from gym import Env #custom environment가능하게 해주는 모듈
from gym.spaces import Discrete, Box #discrete는 이산, box는 continous!
from gym import spaces, logger
import numpy as np
import random

In [19]:
###Simulation settings referred to Table 1
W = 20 * (10 ** 6) #channel bandwidth
I_max = 20 #maximum associated number
D = 10000 #size of replay memory
B_max = 100 #battery capacity
M = 100 #data requests
P_tx = 3 #transceiver power
P_op = 10 #operation power
P__c = 1 #swithcing power
background_noise = -174 #backgroud noise
alpha = 2 #path loss
num_SBS = 5 #the number of SBS
N = list(range(1, num_SBS + 1)) # SBS set # N = [1, 2, 3, 4, 5]
T = 200
t = np.int64(np.linspace(0, T, T)) # Time instant #t = (0, 2, ,,, , 199)
time_interval = 1
K = 200 #batch size
num_episodes = 200
Distance = 50

rho = []
for kk in range(num_SBS):
    k_ = [0 for _ in range(T)]
    for kk_ in range(T):
        k_[kk_] = k_[kk_] + random.randint(0,1)
    rho.append(k_)

###4장. simulation resluts and analysis
#시간 t에 유입되는 사람의 수
#rvs = random variable sampling으로 포아송 분산 무작위 변수 생성
user_come = stats.poisson.rvs(mu = np.sin(t)+1) 

#시간t에 나가는 사람의 수
#자의적 해석(0.7명이 떠난다 = 0명이 떠난다. 1.7명이 떠난다 = 1명이 떠난다)내림 적용
#exponential distribution is also called "negative" exponential distribution!!
user_leave = np.floor(stats.expon.rvs(scale = 1, size = T)) 

#시간 t에 실제로 있는 사람의 수 
num_users = np.cumsum(user_come - user_leave)
#random seed에 따라서 사람의 수가 음수가 될 수 도 있음. 따라서 임의 조건 부여
def num_user():
    for h in range(T):
        if num_users[h] < 0:
            num_users[h] = 0
    return num_users


#energy arrival
#행(SBS) x 열(시간)
e_come = []
for ec in range(num_SBS):
    ece = []
    ece = stats.poisson.rvs(mu = np.cos(t)+1)
    e_come.append(ece)    


###2장. system model
#The battery level of SBS j at time instant t
#아직 rho(state of SBS j at time instant t)는 구현안되었음. DQN알고리즘 적용 후에나 구현가능.
#행(SBS) x  열(t)
B = []
for j in range(num_SBS):
    C = [0 for _ in range(T)]
    for k in range(T):
        C[k] = min(C[k-1] - (P_tx * rho[j][k]) + e_come[j][k], B_max) 
    B.append(C)
    
##Network performance
#User association rule
#50m범위에서 SBS 랜덤 생성
loc_SBS = [] 
for l_SBS in range(num_SBS):
    radius = np.random.uniform(0,Distance)
    radian = random.random()
    x_SBS = radius * math.cos(2 * math.pi * radian)
    y_SBS = radius * math.sin(2 * math.pi * radian)
    loc_SBS.append([x_SBS,y_SBS])
#print(loc_SBS)
    
#시간 t에 따른 user들의 위치(50m 이내)
loc_user = [] 
for l_user_ in range(T):
    loc_user_ = []
    for l_user_ in range(int(num_user()[l_user_])):
        radius_ = np.random.uniform(0,Distance)
        radian_ = random.random()
        x_user = radius_ * math.cos(2 * math.pi * radian_)
        y_user = radius_ * math.sin(2 * math.pi * radian_)
        loc_user_.append([x_user,y_user])
    loc_user.append(loc_user_)
#print(loc_user)

def distance(x1, y1, x2, y2):
    result = math.sqrt(math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
    return result
#di_di = distance between SBS j and user i at time instant t
#시간 x 유저 수 x SBS
di_di = []
for di in range(T): #시간 t일때 사용자 좌표 집합
    dis_dis = []
    for dis in range(int(num_user()[di])):#i번째 사람
        jth_jth = []
        for jth in range(num_SBS): #j번째 SBS에 대하여
            jth_jth.append(distance(loc_SBS[jth][0],loc_SBS[jth][1],loc_user[di][dis][0],loc_user[di][dis][1]))
        dis_dis.append(jth_jth)
    di_di.append(dis_dis)        


#j_star = j star referred in User association rule
#np.array(j_star) : 행(시간t), 열(t에 따른 사람수로 가변적임)에서 각 유저들과 가장 가까운 SBS값
j_star = []
for min_dist_ in range(T):
    Y = [0 for _ in range(int(num_user()[min_dist_]))]
    for Y_ in range(int(num_user()[min_dist_])):
        Y[Y_] = di_di[min_dist_][Y_].index(min(di_di[min_dist_][Y_]))
    j_star.append(Y)        
    
    
def kth_largest_number(arr, K):
    unique_nums = set(arr)
    sorted_nums = sorted(unique_nums, reverse=False) #
    return sorted_nums[K-1]
    
#np.array(j_star) : 행(시간t), 열(t에 따른 사람수로 가변적임)에서 각 유저들과 가장 가까운 SBS값
def j_kth_star(k):
    j_second_star = []
    for sec_dist_ in range(T):
        S = [0 for _ in range(int(num_user()[sec_dist_]))]
        for S_ in range(int(num_user()[sec_dist_])):
            S[S_] = di_di[sec_dist_][S_].index(kth_largest_number(di_di[sec_dist_][S_],k))
        j_second_star.append(S)     
    return j_second_star
    
#I_j_(t) : the set of users associated with the SBS j at time instant t
#행(SBS) x 열(시간 t) : user_set[0][2] = time2일때, SBS 0과 가장 가까운 사람들
#max = 20으로 20초과하면 그 다음 가까운 SBS로 가도록
user_set = []
for user_set_ in range(num_SBS):
    U = [0 for _ in range(T)]
    for U_ in range(T):
        U[U_] = [j_index for j_index, value in enumerate(j_star[U_]) if value == user_set_] 
        #j_star[U_] = t = U_일때 각 유저들과 가까운 SBS값
        if len(U[U_]) > I_max:
            U[U_] = [j_index for j_index, value in enumerate(j_kth_star(2)[U_]) if value == user_set_]  
            if len(U[U_]) > I_max:
                U[U_] = [j_index for j_index, value in enumerate(j_kth_star(3)[U_]) if value == user_set_]
                if len(U[U_]) > I_max:
                    U[U_] = [j_index for j_index, value in enumerate(j_kth_star(4)[U_]) if value == user_set_]
                    if len(U[U_]) > I_max:
                        U[U_] = [j_index for j_index, value in enumerate(j_kth_star(5)[U_]) if value == user_set_] 
    user_set.append(U)    
     
#abs of user set with SBS j at time instant t
#행(SBS) x 열(t)에서의 SBS j에 할당되는 총 유저 숫자
abs_user_set = []
for abs_ in range(num_SBS):
    A = [0 for _ in range(T)]
    for A_ in range(T):
        A[A_] = len(user_set[abs_][A_])
    abs_user_set.append(A)    
      
    
#traffic load of SBS j at time instant t
#행(SBS) x 열(t)에서의 traffic load
traffic_load = []
for traffic_load_ in range(num_SBS):
    TL = [0 for _ in range(T)]
    for TL_ in range(T):
        TL[TL_] = abs_user_set[traffic_load_][TL_] / I_max
    traffic_load.append(TL)
    
#SINR에서 분자 signal에 해당하는 값
#행(t) x 열(user, i) x j(SBS) 의 3차원 행렬 
#signal[1][0] #t = 1일 때, 0th 유저와 5개의 SBS 신호세기
signal = []
for sig in range(T): #시간 t일때 j,i간의 신호세기
    sig_ = []
    for s in range(int(num_user()[sig])):#t일때 모든 j에 대하여
        s_ = []
        for si in range(num_SBS): #j번째 SBS에 대하여
            s_.append((P_tx) * (np.exp(1) * rho[si][sig] * (di_di[sig][s][si])))
        sig_.append(s_)
    signal.append(sig_)          
    
#signal 전체 합
signal_deno = []
for noi in range(T):
    signal_denominator = [0 for _ in range(int(num_user()[noi]))]
    for noi_ in range(int(num_user()[noi])):
        for noi_SBS in range(num_SBS):
            signal_denominator[noi_] = signal_denominator[noi_] + signal[noi][noi_][noi_SBS]
    signal_deno.append(signal_denominator)

    
#SINR에서 분모 중 Interference 값
interference = background_noise * W

#SINR
#t x i(user) x j(SBS)
#sinr[10][0][0] : t = 10일때 user0과 sbs0간의 sinr비
sinr = []
for sinrt in range(T): #시간 t일때 (i,j)의 sinr
    sinr_t = []
    for sinri in range(int(num_user()[sinrt])):#i번째 유저와의 SINR값은
        sinr_t_j = []
        for sinrj in range(num_SBS): #j번째 SBS에 대하여
            sinr_t_j.append((signal[sinrt][sinri][sinrj]) / ((signal_deno[sinrt][sinri] - (signal[sinrt][sinri][sinrj])) + interference))
        sinr_t.append(sinr_t_j)
    sinr.append(sinr_t)    
    
#Throughput when user i associated to SBS j at time instant t
#throughput[5][6][2] : t=5, user6, sbs2일때 throughput
throughput = []
for throughputt in range(T):
    throughput_t = []
    for throughputi in range(int(num_user()[throughputt])):
        throughput_t_j = []
        for throughputj in range(num_SBS): #j번째 SBS에 대하여
            throughput_t_j.append(time_interval * W * np.log2(1 + sinr[throughputt][throughputi][throughputj]))
        throughput_t.append(throughput_t_j)
    throughput.append(throughput_t)  
    
#total throughput achieved by the SBS j at time instant t'
#total_throughput[3][0] : SBS = 3, t = 0에서의 total throughput
total_throughput = []
for total_t in range(num_SBS):
    total_throughput_ = [0 for _ in range(T)]
    for total_jt_ in range(T):
        for total_jt in range(int(num_user()[total_jt_])):
            total_throughput_[total_t] = total_throughput_[total_t] + throughput[total_jt_][total_jt][total_t]
    total_throughput.append(total_throughput_)
    
#The achievable data rate of user i served by SBS j at time t
#data_rate[8][0][3] : t=8, user 0, SBS 3에서의 achievable data rate
data_rate = []
for dt in range(T):
    data_rate_ = []
    for dy in range(int(num_user()[dt])):
        data_rate__ = []
        for dz in range(num_SBS): #j번째 SBS에 대하여
            data_rate__.append(W * throughput[dt][dy][dz])
        data_rate_.append(data_rate__)
    data_rate.append(data_rate_)  

#total traffic dealy of SBS j at time instant t
#traffic_delay[4][3] : SBS = 4, t = 3에서의 total traffic delay
traffic_delay = []
for ddt in range(num_SBS):
    traffic_delay_ = [0 for _ in range(T)]
    for ddy in range(T):
        for ddz in range(int(num_user()[ddy])):
            traffic_delay_[ddt] = traffic_delay_[ddt] + (100 / data_rate[ddy][ddz][ddt])
    traffic_delay.append(traffic_delay_)
    
#total delay
#total_delay[0] t=0일때 모든 SBS j에 대한 total delay
total_delay = []
for a,b,c,d,e in zip(traffic_delay[0], traffic_delay[1], traffic_delay[2], traffic_delay[3], traffic_delay[4]):
    total_delay.append(a + b + c + d + e)
    

##Energy Consumption
#SBS j's power consumption of serving users at time t
#power_serving[3][4] : SBS 3이 t = 4에 power consumption
power_serving = []
for pst in range(num_SBS):
    power_serving_ = [0 for _ in range(T)]
    for psy in range(T):
        power_serving_[psy] = ((traffic_load[pst][psy] * P_op) + P_tx) * rho[pst][psy]
    power_serving.append(power_serving_)

    
#power consumption of swithing states of SBS j
#beta[2][5] : SBS 2, t = 5에서 beta값(transition이 일어났는가?)
beta = []
for betaj in range(num_SBS):
    beta_ = [0 for _ in range(T)]
    for betak in range(T):
        if rho[betaj][betak] + rho[betaj][betak - 1] == 1: #position transition
            beta_[betak] =  1
        else:
            beta_[betak] = 0
    beta.append(beta_)

#power consumption of state switch
power_switch = []
for pswx in range(num_SBS):
    power_switch_ = [0 for _ in range(T)]
    for pswy in range(T):
        power_switch_[pswy] = beta[pswx][pswy] * P__c
    power_switch.append(power_switch_)
    
#total_power_consumption[3][2] : SBS 3과 t = 2에서 파워 소모량
total_power_consumption = []
for tpcj in range(num_SBS):
    summation = []
    summation = [w + e for w, e in zip(power_switch[tpcj], power_serving[tpcj])]
    total_power_consumption.append(summation)
    
#total_power_consumption[5] : t=5에서 total power consumption
to_pow_consum = [0 for _ in range(T)]
for topo in range(T):
    for topo_ in range(num_SBS):
        to_pow_consum[topo] = to_pow_consum[topo] + total_power_consumption[topo_][topo]
        
#total throughput at all j
to_throughput = [0 for _ in range(T)]
for tot in range(T):
    for toth in range(num_SBS):
        to_throughput[tot] = to_throughput[tot] + total_throughput[toth][tot]
        
#EE(energy efficiency) at time instant t
ee = []
for eet in range(T):
    ee = [t / c for t, c in zip(to_throughput, to_pow_consum)]
    
###3장. Problem formulation
##Optimization problem formulation
#The utility funtion G(t)
utility = []
zeta = 0.5
for utt in range(T):
    utility = [(zeta * a) - ((1 - zeta) * b) for a, b in zip(ee, total_delay)]
    
discount_factor = 0.01
#optimization problem
#t부터 T(200)까지 Utility func 다 구함 with discount factor
def op(t):
    opm = utility[t] * discount_factor
    for k in range(t + 1,T):
        opm = opm + utility[k] * (discount_factor ** (k+1))
    return opm

##DQN framework formulation
#state_space 탄생
#t기준으로 맞추기 위해 전치
state_e = np.array(e_come).T #t x j
state_b = np.array(B).T #t x j
state_l = np.array(traffic_load).T #t x j
state_r = np.array(total_throughput).T #t x j
state_d = np.array(traffic_delay).T #t x j
#state_space[0] : t = 0일 때, ((e,b,l,r,d)_j1, (e,b,l,r,d) -j2, ,,, (e,b,l,r,d)_j5)
state_space = [list(zip(x, y, z, l, k)) for x, y, z, l, k in zip(state_e, state_b, state_l, state_r, state_d)]

#action_space 탄생
#t기준으로 맞추기 위해 전치
action_space = np.array(rho).T

#reward function = "utility"(utility function G(t))

ZeroDivisionError: float division by zero

In [11]:
class CustomEnv:
    def __init__(self): 
        #환경시작할 때 환경 초기값 설정
        self.num_SBS = 5
        self.W = 20 * (10 ** 6) #channel bandwidth
        self.I_max = 20 #maximum associated number
        self.D = 10000 #size of replay memory
        self.B_max = 100 #battery capacity
        self.M = 100 #data requests
        self.P_tx = 3 #transceiver power
        self.P_op = 10 #operation power
        self.P__c = 1 #swithcing power
        self.background_noise = -174 #backgroud noise
        self.alpha = 2 #path loss
        self.num_SBS = 5 #the number of SBS
        self.N = list(range(1, self.num_SBS + 1)) # SBS set # N = [1, 2, 3, 4, 5]
        self.T = 200
        self.t = np.int64(np.linspace(0, self.T, self.T)) # Time instant #t = (0, 2, ,,, , 199)
        self.time_interval = 1
        self.K = 200 #batch size
        self.num_episodes = 200
        self.Distance = 50
        
        self.step_count = 0
        self.num_user = 0
        self.num_actions = 2 ** (self.num_SBS) # = 32
      
    ###action : SBS 5개, action = 0,1
    ###mapping = {
        #          5 4 3 2 1             5 4 3 2 1
        #     0 : [0,0,0,0,0] #     1 : [0,0,0,0,1]
        #     2 : [0,0,0,1,0]#     3 : [0,0,0,1,1]
        #     4 : [0,0,1,0,0]#     5 : [0,0,1,0,1]
        #     6 : [0,0,1,1,0]#     7 : [0,0,1,1,1]
        #     8 : [0,1,0,0,0]#     9 : [0,1,0,0,1]
        #    10 : [0,1,0,1,0]#    11 : [0,1,0,1,1]
        #    12 : [0,1,1,0,0]#    13 : [0,1,1,0,1]
        #    14 : [0,1,1,1,0]#    15 : [0,1,1,1,1]
        #    16 : [1,0,0,0,0]#    17 : [1,0,0,0,1]
        #    18 : [1,0,0,1,0]#    19 : [1,0,0,1,1]
        #    20 : [1,0,1,0,0]#    21 : [1,0,1,0,1]
        #    22 : [1,0,1,1,0]#    23 : [1,0,1,1,1]
        #    24 : [1,1,0,0,0]#    25 : [1,1,0,0,1]
        #    26 : [1,1,0,1,0]#    27 : [1,1,0,1,1]
        #    28 : [1,1,1,0,0]#    29 : [1,1,1,0,1]
        #    30 : [1,1,1,1,0]#    31 : [1,1,1,1,1]
        ###}###
        
        
        bounds_lower = np.array([
            0,
            0,
            0,
            0,
            0,#SBS_1의 total traffic delay min값,
            0,
            0,
            0,
            0,
            0,#SBS_1의 total traffic delay min값,
            0,
            0,
            0,
            0,
            0,#SBS_1의 total traffic delay min값,
            0,
            0,
            0,
            0,
            0,#SBS_1의 total traffic delay min값,
            0,
            0,
            0,
            0,
            0,#SBS_1의 total traffic delay min값,
            ])

        bounds_upper = np.array([
            7,
            self.B_max,
            1,
            99999999,#SBS_1의 total throughput max값,
            99999999,#SBS_1의 total traffic delay max값,
            7,
            self.B_max,
            1,
            99999999,#SBS_2의 total throughput max값,
            99999999,#SBS_2의 total traffic delay max값,
            7,
            self.B_max,
            1,
            99999999,#SBS_3의 total throughput max값,
            99999999,#SBS_3의 total traffic delay max값,
            7,
            self.B_max,
            1,
            99999999,#SBS_4의 total throughput max값,
            99999999,#SBS_4의 total traffic delay max값,
            7,
            self.B_max,
            1,
            99999999,#SBS_5의 total throughput max값,
            99999999,#SBS_5의 total traffic delay max값
            ])

        self.action_space = spaces.Discrete(self.num_actions) # action size is here
        self.observation_space = spaces.Box(bounds_lower, bounds_upper, dtype=np.float32) # spaces.Discrete(2) # state size is here 
        
        self.state = None
    
    def reset(self):
        self.state = [0,0,0,0,
                      #?,#delay_SBS_1 초기값 별도 설명 없음.임의로 초기화해야할듯
                      0,0,0,0,
                      #?,#delay_SBS_2 초기값 별도 설명 없음.임의로 초기화해야할듯
                      0,0,0,0,
                      #?,#delay_SBS_3 초기값 별도 설명 없음.임의로 초기화해야할듯
                      0,0,0,0,
                      #?,#delay_SBS_4 초기값 별도 설명 없음.임의로 초기화해야할듯
                      0,0,0,0,
                      #?,#delay_SBS_5 초기값 별도 설명 없음.임의로 초기화해야할듯
                     ]        
        self.num_user = 0
        return self.state
    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        self.num_user += self.num_users() #해당 step의 user 수
        reward = 0
        e_1, b_1, l_1, th_1, d_1, e_2, b_2, l_2, th_2, d_2, e_3, b_3, l_3, th_3, d_3, e_4, b_4, l_4, th_4, d_4, e_5, b_5, l_5, th_5, d_5 = state
        self.step_count += 1
        # Apply action
        # user의 변화
        # solution 찾았는가?
        # Update step
        i = np.random.randint(0,200,num_SBS)
        for p in range(self.num_SBS):
            state[p * 5] = stats.poisson.rvs(mu = np.cos(self.t)+1)[i][p] #state 중 e를 update
            state[p * 5 + 1] = self.battery_level(action)[p] #state 중 b를 update
            state[p * 5 + 2] = self.traffic_load(self.num_user)[0][p] #state 중 l를 update
            state[p * 5 + 3] = self.total_throughput(self.num_user, action)[1][p]
            state[p * 5 + 4] = self.battery_level()

        
        # Ruduce shower length by 1 second    
        # Calculate reward       
        # Check if shower is done
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def render(self):
        # Implement visualization
        pass
 ##############################################################################################################   
    def battery_level(self, action):
        battery_1 = min(state[1] - (self.P_tx * ((action & 0b00001) / 0b00001)) + state[0], self.B_max)
        battery_2 = min(state[6] - (self.P_tx * ((action & 0b00010) / 0b00010)) + state[5], self.B_max)
        battery_3 = min(state[11] - (self.P_tx * ((action & 0b00100) / 0b00100)) + state[10], self.B_max)
        battery_4 = min(state[16] - (self.P_tx * ((action & 0b01000) / 0b01000)) + state[15], self.B_max)
        battery_5 = min(state[21] - (self.P_tx * ((action & 0b10000) / 0b10000)) + state[20], self.B_max)
        return battery_1, battery_2, battery_3, battery_4, battery_5
    
    def num_users(self):#1step user 증가량
        user_come = stats.poisson.rvs(mu = np.sin(self.t)+1)[np.random.randint(0,200)] 
        user_leave = np.floor(stats.expon.rvs(scale = 1, size = self.T))[np.random.randint(0,200)] 
        num_users = user_come - user_leave
        if num_users < 0:
            num_users = 0
        return num_users 
    
    def traffic_load(self, num_user):
        #SBS의 위치 정하기
        loc_SBS = [] 
        for l_SBS in range(num_SBS):
            radius = np.random.uniform(0,self.Distance)
            radian = random.random()
            x_SBS = radius * math.cos(2 * math.pi * radian)
            y_SBS = radius * math.sin(2 * math.pi * radian)
            loc_SBS.append([x_SBS,y_SBS])
        
        #User의 위치 정하기
        loc_user = []
        for l in range(num_user):
            radius_ = np.random.uniform(0,self.Distance)
            radian_ = random.random()
            x_user = radius_ * math.cos(2 * math.pi * radian_)
            y_user = radius_ * math.sin(2 * math.pi * radian_)
            loc_user.append([x_user,y_user])
        
        def distance(x1, y1, x2, y2):
            result = math.sqrt(math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
            return result

        dis_dis = []
        for dis in range(num_user):#i번째 사람
            jth_jth = []
            for jth in range(num_SBS): #j번째 SBS에 대하여
                jth_jth.append(distance(loc_SBS[jth][0],loc_SBS[jth][1],loc_user[dis][0],loc_user[dis][1]))
            dis_dis.append(jth_jth)
 
        j_star = [0 for _ in range(num_user)]
        for y in range(int(num_user)):
            j_star[y] = dis_dis[y].index(min(dis_dis[y]))
        
        def kth_largest_number(arr, K):
            unique_nums = set(arr)
            sorted_nums = sorted(unique_nums, reverse=False) #
            return sorted_nums[K-1]
        
        def j_kth_star(k):
            j_kth_star = []
            j_second_star = [0 for _ in range(num_user)]
            for S_ in range(num_user):
                j_second_star[S_] = dis_dis[S_].index(kth_largest_number(dis_dis[S_], k))
            j_kth_star.append(j_second_star)     
            return j_kth_star

        user_set = [0 for _ in range(num_SBS)]
        for u in range(num_SBS):
                user_set[u] = [j_index for j_index, value in enumerate(j_star) if value == u] 
                if len(user_set[u]) > I_max:
                    user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(2)) if value == u]  
                    if len(user_set[u]) > I_max:
                        user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(3)) if value == u]
                        if len(user_set[u]) > I_max:
                            user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(4)) if value == u]
                            if len(user_set[u]) > I_max:
                                user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(5)) if value == u] 

        abs_user_set = [0 for _ in range(num_SBS)]
        for a in range(num_SBS):
            abs_user_set[a] = len(user_set[a])
        
        traffic_load = [0 for _ in range(num_SBS)]
        for t in range(num_SBS):
            traffic_load[t] = abs_user_set[t] / I_max

        return np.array(traffic_load), np.array(user_set)

    def total_throughput(self, num_user, action):
        user_set = self.traffic_load(num_user)[1]
        ##분모의 rho_i_t 구현
        act = []
        for o in range(num_user):
            if o in u_set[0]:
                c = ((action & 0b00001) / 0b00001)
            elif o in u_set[1]:
                c = ((action & 0b00010) / 0b00010)
            elif o in u_set[2]:
                c = ((action & 0b00100) / 0b00100)
            elif o in u_set[3]:
                c = ((action & 0b01000) / 0b01000)
            elif o in u_set[4]:
                c = ((action & 0b10000) / 0b10000)
            act.append(c)
                
        ##sinr의 분자
        signal = []
        #for s in range(num_user):  #t일때 모든 j에 대하여
        for s in range(num_user):   
            s_ = []
            for si in range(self.num_SBS):
                s_.append((self.P_tx) * (np.exp(1) * act[s] * (dis_dis[s][si])))            
            signal.append(s_)
        
        ##sinr의 분모 path loss 적용
        signal_alpha = []
        for s in range(num_user):   
            ph = []
            for si in range(self.num_SBS):
                ph.append((self.P_tx) * (np.exp(1) * act[s] * (dis_dis[s][si] ** (-self.alpha))))            
            signal_alpha.append(ph)
    
        ##sinr의 분모2 - 전체더하기
        signal_denominator = [0 for _ in range(num_user)]
        for n_ in range(num_user):
            for no in range(self.num_SBS):
                signal_denominator[n_] = signal_denominator[n_] + signal_alpha[n_][no]
   
        sinr = []
        for z in range(num_user):
            sinr_ = []
            for q in range(self.num_SBS):
                sinr_.append((signal[z][q]) / ((signal_denominator[z] - (signal_alpha[z][q])) + (2 * 10 **(-13.4))))
            sinr.append(sinr_)
    
        throughput = []
        for i in range(num_user):
            t_t_j = []
            for j in range(self.num_SBS): 
                t_t_j.append(self.time_interval * self.W * np.log2(1 + sinr[i][j]))
            throughput.append(t_t_j)
        
        total_throughput = [0 for _ in range(self.num_SBS)]
        for t in range(self.num_SBS):
            for j in range(num_user):
                total_throughput[t] = total_throughput[t] + throughput[j][t]

        return np.array(throughput), np.array(total_throughput)

In [12]:
env = CustomEnv()

In [13]:
env.step(1)

TypeError: cannot unpack non-iterable NoneType object

In [17]:
#def total_traffic_delay(self):

def total_traffic_delay():
    #th = self.total_throughput(16, 3)[0]
    th = total_throughput(16, 3)[0]
    data_rate = []
    for y in range(num_user):
        data_rate_ = []
        #for z in range(self.num_SBS): 
        for z in range(num_SBS): 
            data_rate_.append(W * th[y][z])
            #data_rate_.append(self.W * th[y][z])
    data_rate.append(data_rate_)  
    
    return data_rate
total_traffic_delay()

NameError: name 'self' is not defined

In [ ]:
#The achievable data rate of user i served by SBS j at time t
#data_rate[8][0][3] : t=8, user 0, SBS 3에서의 achievable data rate
data_rate = []
for dt in range(T):
    data_rate_ = []
    for dy in range(int(num_user()[dt])):
        data_rate__ = []
        for dz in range(num_SBS): #j번째 SBS에 대하여
            data_rate__.append(W * throughput[dt][dy][dz])
        data_rate_.append(data_rate__)
    data_rate.append(data_rate_)  

#total traffic dealy of SBS j at time instant t
#traffic_delay[4][3] : SBS = 4, t = 3에서의 total traffic delay
traffic_delay = []
for ddt in range(num_SBS):
    traffic_delay_ = [0 for _ in range(T)]
    for ddy in range(T):
        for ddz in range(int(num_user()[ddy])):
            traffic_delay_[ddt] = traffic_delay_[ddt] + (100 / data_rate[ddy][ddz][ddt])
    traffic_delay.append(traffic_delay_)
    
#total delay
#total_delay[0] t=0일때 모든 SBS j에 대한 total delay
total_delay = []
for a,b,c,d,e in zip(traffic_delay[0], traffic_delay[1], traffic_delay[2], traffic_delay[3], traffic_delay[4]):
    total_delay.append(a + b + c + d + e)
 